# <div style="text-align: center"> <font size=+5> <ins>PREDICT ANSWERS from INDICATORS</ins> </font> </div>
___


### Imports and settings

In [1]:
#imports 

import numpy as np
import pandas as pd

import os

import ipywidgets as widgets



import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt



from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


import tensorflow as tf

import statsmodels as sm

2024-10-08 11:28:27.715044: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#settings

pd.set_option('display.max_columns', 500)

plt.rcParams.update({'figure.dpi' : 300,
                     'mathtext.fontset' : 'stix', 
                     'font.family' : 'STIXGeneral'
                    })

___
## Get datasets

In [3]:
# get full dataset
data_issp = pd.read_pickle("../Processed_Data/questionnaires/issp_questionnaires_common.pickle")

# convert values to numerical data
conv_dict = {
    "Respondent Data" : {
        "Sex" : {"Male" : -1, "Female" : 1},
        "Education level" : {"No degree" : 0, "Primary" : 1, "Secondary" : 2, "University" : 3},
        "Living area" : {"Rural area" : 0, "Small city" : 1, "Big city" : 2},
        "Political orientation" : {"Far Left" : -3, "Left" : -2, "Center Left" : -1, "Center" : 0, "None/Other" : 0, "Center Right" : 1, "Right" : 2, "Far Right" : 3},
        "Religious beliefs" : {"Atheist" : 0, "Christian" : 1, "Islamic" : 1, "Jewish" : 1, "Other" : 1}
    },
    "Questions" : {
        "Strongly Disagree" : -2,
        "Disagree" : -1,
        "Neither Agree nor Disagree" : 0,
        "Agree" : 1,
        "Strongly Agree" : 2,

        "Yes" : 1,
        "No" : -1,

        'Extremely dangerous': 2,
        'Very dangerous': 1,
        'Somewhat dangerous': 0,
        'Not very dangerous': -1,
        'Not dangerous at all': -2,

        "Always" : 3,
        "Often" : 2,
        "Sometimes" : 1,
        "Never" : 0,

        'Very willing': 2,
        'Fairly willing': 1,
        'Neither willing nor unwilling': 0,
        'Fairly unwilling': -1,
        'Very unwilling': -2
    }
}

#convert values to numerical
data_resp = data_issp.loc[:,("Respondent Data", slice(None))].apply(lambda col: col if col.name[1] not in conv_dict["Respondent Data"] else col.map(lambda x: np.nan if pd.isna(x) else conv_dict["Respondent Data"][col.name[1]][x]).astype(float))
data_quest = data_issp.loc[:,("Questions", slice(None))].map(lambda x: conv_dict["Questions"].get(x, x)).astype(float)

#cluster questions based on topic
clusters = {
    #TOPIC 1: awareness of the environmental consequences of societies’ modern, industrial activities
    "Awareness" : [
        'Danger to the environment (Air pollution by cars)',
        'Danger to the environment (Air pollution by industry)',
        'Danger to the environment (Pesticides and chemicals in farming)',
        'Danger to the environment (River, lake and stream pollution)',
        'Danger to the environment (Rise in the world’s temperature)',
        'Danger to the environment (Nuclear power stations)'
    ],

    #TOPIC 2: willingness to sacrifice personally in some manner (e.g. time, money) for the environment
    "Will to make sacrifices" : [
        'Willingness to Make Trade-Offs for Environment (Pay much higher prices)',
        'Willingness to Make Trade-Offs for Environment (Pay much higher taxes)',
        'Willingness to Make Trade-Offs for Environment (Cut your standard of living)',
        'Member of a group to preserve environment',
        'In the last five years, signed a petition',
        'In the last five years, given money',
        'In the last five years, participated in an environmental demonstration',
        'I do what is right even when it costs money and takes time'
    ],
    
    #TOPIC 3: importance of addressing environmental problems and of collective effort for resolving environmental
    #issues and how environmental issues intersect with economic issues, science and progress
    # NB!! answers are rescaled so that higher scores reflect pro-environment responses
    "Efficacy of environmental action" : [
        'We worry too much about harming environment',
        'We worry too much about future environment',
        'Science solves environmental problems',
        'Too difficult to do much about environment'
    ]
        
}

#rescale
data_to_cluster = data_quest.apply(lambda x: x if x.name[1] not in clusters["Efficacy of environmental action"] else -x)
data_quest_clustered = pd.concat([pd.concat([data_to_cluster.loc[:, ("Questions", clusters[topic])].sum(axis=1) for topic in clusters], axis=1, keys=clusters.keys())], keys=["Environmental concern dimension"],  names=["Variable Type", "Variable Name"], axis=1)


#get country-wide indicators columns
macro_indicators_dict = {(ind[:-7]) : pd.read_pickle(f"Processed_Data/indicators/{ind}") for ind in os.listdir("../Processed_Data/indicators") if ind[-7:] == ".pickle"}
data_indic = pd.concat([pd.DataFrame.from_records([data_issp.index.map(lambda idx: indicator.loc[idx[1], idx[0]]) for indicator in macro_indicators_dict.values()], index=macro_indicators_dict.keys(), columns=data_issp.index).T], keys=["Country-wide indicators"], names=["Variable Type", "Variable Name"], axis=1)


#merge into single dataframes
data_issp_numerical = pd.concat([data_resp, data_indic, data_quest], axis=1) #separate questions
data_issp_numerical_clustered = pd.concat([data_resp, data_indic, data_quest_clustered], axis=1) #clustered questions


del(data_issp, data_resp, data_quest, data_to_cluster, data_quest_clustered, data_indic, conv_dict)


#dump
#data_issp_numerical.to_pickle("../Saved_variables/full_dataset_numerical_quest+indic.pickle")
#data_issp_numerical_clustered.to_pickle("../Saved_variables/full_dataset_numerical_clustered.pickle")

In [3]:
#quick load
data_issp_numerical = pd.read_pickle("../Saved_variables/full_dataset_numerical_quest+indic.pickle")
data_issp_numerical_clustered = pd.read_pickle("../Saved_variables/full_dataset_numerical_clustered.pickle")

___

In [4]:
#rename macro indicators columns
newcols = {
    "GHG_emissions"     : "Greenhouse gases emissions",
    "PM25_exposure"     : "PM25 exposure",
    "GDP_growth"        : "GDP growth",
    "gini_index"        : "Gini index",
    "GDP_pcap_ppp"      : "GDP per capita (Purchasing Power Parity)",
    "water_withdrawals" : "Water withdrawals",
    "pesticides_use"    : "Pesticides use",
    "GDP_pcap"          : "GDP per capita",
    "internet_usage"    : "Internet usage",
    "CO2_emissions_per_capita" : "CO2 emissions per capita",
    "infant_mortality"  : "Infant Mortality"
          }

newcols = {("Country-wide indicators", key) : ("Country-wide indicators", value) for key, value in newcols.items()}

data_issp_numerical.columns = data_issp_numerical.columns.map(lambda x: newcols.get(x,x))
data_issp_numerical_clustered.columns = data_issp_numerical_clustered.columns.map(lambda x: newcols.get(x,x))

___

In [5]:
#group by year and country (weighted mean)
data_issp_numerical_grouped = data_issp_numerical.drop(columns=("Respondent Data", "Weight")).mul(data_issp_numerical.loc[:,("Respondent Data", "Weight")], axis=0).groupby(["Year", "Country"]).sum(min_count=1).div(data_issp_numerical.loc[:,("Respondent Data", "Weight")].groupby(["Year", "Country"]).sum(), axis=0)
data_issp_numerical_clustered_grouped = data_issp_numerical_clustered.drop(columns=("Respondent Data", "Weight")).mul(data_issp_numerical_clustered.loc[:,("Respondent Data", "Weight")], axis=0).groupby(["Year", "Country"]).sum(min_count=1).div(data_issp_numerical_clustered.loc[:,("Respondent Data", "Weight")].groupby(["Year", "Country"]).sum(), axis=0)

In [6]:
#rescale so that dimensions columns are in range [-1,+1]
data_issp_numerical_clustered.loc[:,("Environmental concern dimension", slice(None))] /= [12,12,8]
data_issp_numerical_clustered_grouped.loc[:,("Environmental concern dimension", slice(None))] /= [12,12,8]

___
## Lasso

In [4]:
def fit_lasso(yX, alpha, weights=None):
    model = Lasso(alpha)
    model.fit(yX.iloc[:,1:], yX.iloc[:,0], sample_weight=weights)
    return {"alpha" : alpha, "loss" : 1-model.score(yX.iloc[:,1:], yX.iloc[:,0], sample_weight=weights), "coefficients" : model.coef_, "model" : model}

def plot_indicators(data, question, year, indicators):
    yX = data.loc[(year, slice(None)),[("Environmental concern dimension", question), *indicators]].dropna().astype(float)
    yX.iloc[:,:] = StandardScaler().fit_transform(yX.to_numpy())
    
    try:
        weights = data.loc[yX.index, ("Respondent Data", "Weight")]
    except:
        weights = None
    
    res = pd.DataFrame([fit_lasso(yX, alpha, weights) for alpha in np.logspace(-2,0,250)])
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(6.4*2,4.8), gridspec_kw={"width_ratios" : (5,95)}, sharey=True)
    fig.subplots_adjust(wspace=0)
    fig.suptitle(f"Most important indicators to predict '{question}' in {year}", fontsize=20)
    data = np.stack(res["coefficients"].to_numpy())
    sns.heatmap([[np.log(l)] for l in res["loss"]], ax=ax1, cbar=False)
    sns.heatmap(data, cmap="bwr", center=0, ax=ax2)
    ax2.set_title("Coefficients in LASSO regression")
    ax2.set_xticklabels([col[1] for col in yX.iloc[:,1:].columns], rotation=50, horizontalalignment="right", fontsize=8)
    ax2.set_xlabel("Indicator")

    for ax in [ax1]:
        ax.set_yticks(res.index[::50])
        ax.set_yticklabels([f"{l:.0e}" for l in res.loc[::50,"alpha"]], rotation=30, horizontalalignment="right", verticalalignment="top", fontsize=8)
        ax.tick_params(axis="y", pad=0)
        ax.set_ylabel("L1 regularization hyperparameter")
    
    ax1.set_title("Model error")
    ax1.set_xticks([])
    
    plt.tight_layout()

### Macro Indicators

In [6]:
@widgets.interact(year=[1993,2000,2010,2020], dimension=data_issp_numerical_clustered.loc[:,"Environmental concern dimension"].columns)
def plot(dimension, year=2020):
    indicators = [("Country-wide indicators", ind) for ind in ["GHG_emissions", "PM25_exposure", "GDP_growth", "gini_index", "GDP_pcap_ppp", "water_withdrawals", "pesticides_use"]]
    plot_indicators(data_issp_numerical_clustered_grouped, dimension, year, indicators)

interactive(children=(Dropdown(description='dimension', options=('Awareness', 'Will to make sacrifices', 'Effi…

### Individual indicators

In [7]:
@widgets.interact(year=[1993,2000,2010,2020], dimension=data_issp_numerical_clustered.loc[:,"Environmental concern dimension"].columns, all_countries=False, country=data_issp_numerical_clustered.index.get_level_values("Country").unique())
def plot(dimension, country, all_countries, year=2020):
    indicators = [("Respondent Data", ind) for ind in data_issp_numerical.loc[:, "Respondent Data"].drop(columns="Weight").columns]
    if all_countries:
        plot_indicators(data_issp_numerical_clustered, dimension, year, indicators)
    else:
        plot_indicators(data_issp_numerical_clustered.query("Country == @country"), dimension, year, indicators)

interactive(children=(Dropdown(description='dimension', options=('Awareness', 'Will to make sacrifices', 'Effi…

___
## Linear Fit

In [7]:
def fit_linear(yX, weights=None):
        
    #fill missing values with columns average
    yX = yX.fillna(yX.mean())
    #if whole column is missing fill with 0, then replace with NaN in output
    NaN_cols = yX.iloc[:,1:].mean().isna().to_numpy()
    yX = yX.fillna(0)

    #for large datasets: single train-test split
    if len(yX) > 500:
        yX_train, yX_test = train_test_split(yX, test_size=0.2)

        if weights is not None:
            weights_train, weights_test = weights.loc[yX_train.index], weights.loc[yX_test.index]
        else:
            weights_train, weights_test = None, None
        
        #standardize data
        scaler = StandardScaler()
        yX_train.iloc[:,:] = scaler.fit_transform(yX_train)
        yX_test.iloc[:,:] = scaler.transform(yX_test)
        
        #fit linear regression model
        model = LinearRegression(fit_intercept=True)
        model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)
        
        #compute metrics
        yX_pred = pd.DataFrame().reindex_like(yX_test)
        yX_pred.iloc[:,0] = model.predict(yX_test.iloc[:,1:])
        yX_pred.iloc[:,:] = scaler.inverse_transform(yX_pred)
        yX_test.iloc[:,:] = scaler.inverse_transform(yX_test)
        
        r2_loss = r2_score(yX_test.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights_test)
        mae = mean_absolute_error(yX_test.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights_test)
        

        return {"coefficients" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.coef_)], "losses" : [r2_loss, mae]}


    #for small datasets, leave one out cross validation
    else:
        
        yX_pred = pd.DataFrame().reindex_like(yX) #prepare dataframe for predictions
        
        for idx_num, idx_val in enumerate(yX.index):
            yX_train, yX_test = yX.loc[yX.index != idx_val, :], yX.loc[[idx_val], :]

            if weights is not None:
                weights_train = weights.loc[weights.index != idx_val]
            else:
                weights_train = None

            #standardize data
            scaler = StandardScaler()
            yX_train.iloc[:,:] = scaler.fit_transform(yX_train)
            yX_test.iloc[:,:] = scaler.transform(yX_test)

            #fit linear regression model
            model = LinearRegression(fit_intercept=True)
            model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

            #get predictions
            yX_pred.iloc[idx_num,0] = model.predict(yX_test.iloc[:,1:])

        #rescale back to original predictions data
        yX_pred.iloc[:,:] = scaler.inverse_transform(yX_pred)

        #compute metrics
        r2_loss = r2_score(yX.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights)
        mae = mean_absolute_error(yX.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights)
        
        return {"coefficients" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.coef_)], "losses" : [r2_loss, mae]}

In [ ]:
#get indicators dataframes


##############################################################################################################################################################################################
indicators_macro = [("Country-wide indicators", ind) for ind in ["GHG_emissions", "PM25_exposure", "GDP_growth", "gini_index", "GDP_pcap_ppp", "water_withdrawals", "pesticides_use"]]
predictive_indicators_macro_linearFit  = pd.DataFrame(columns=[("Indicators",ind) for _, ind in indicators_macro]+[("Loss","r2_score"),("Loss", "mae")]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered_grouped.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):    
    data = data_issp_numerical_clustered_grouped.loc[(year, slice(None)), [dimension, *indicators_macro]]
    res = fit_linear(data)
    predictive_indicators_macro_linearFit [(dimension[1], year)] = [*res["coefficients"], *res["losses"]]

predictive_indicators_macro_linearFit = predictive_indicators_macro_linearFit.T
predictive_indicators_macro_linearFit.index = pd.MultiIndex.from_tuples(predictive_indicators_macro_linearFit.index, names=["Environmental concern dimension", "Year"])
predictive_indicators_macro_linearFit.columns = pd.MultiIndex.from_tuples(predictive_indicators_macro_linearFit.columns)

##############################################################################################################################################################################################
indicators_individual = [("Respondent Data", ind) for ind in data_issp_numerical.loc[:, "Respondent Data"].drop(columns="Weight").columns]
predictive_indicators_individual_linearFit = pd.DataFrame(columns=[("Indicators", ind) for _, ind in indicators_individual]+[("Loss","r2_score"),("Loss", "mae")]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):
    for country in data_issp_numerical_clustered.loc[year].index.get_level_values("Country").unique():
        data = data_issp_numerical_clustered.loc[(year, country, slice(None)), [dimension, *indicators_individual]]
        weights = data_issp_numerical_clustered.loc[(year, country, slice(None)), ("Respondent Data", "Weight")]
        res = fit_linear(data, weights)
        predictive_indicators_individual_linearFit[(dimension[1], year, country)] = [*res["coefficients"], *res["losses"]]
        
predictive_indicators_individual_linearFit = predictive_indicators_individual_linearFit.T
predictive_indicators_individual_linearFit.index = pd.MultiIndex.from_tuples(predictive_indicators_individual_linearFit.index, names=["Environmental concern dimension", "Year", "Country"])
predictive_indicators_individual_linearFit.columns = pd.MultiIndex.from_tuples(predictive_indicators_individual_linearFit.columns)



##############################################################################################################################################################################################
#dump results

#predictive_indicators_macro_linearFit.to_pickle("../Saved_variables/predictive_indicators_macro_linearFit.pickle")
#predictive_indicators_individual_linearFit.to_pickle("../Saved_variables/predictive_indicators_individual_linearFit.pickle")

In [15]:
#quick load

predictive_indicators_macro_linearFit = pd.read_pickle("../Saved_variables/predictive_indicators_macro_linearFit.pickle")
predictive_indicators_individual_linearFit = pd.read_pickle("../Saved_variables/predictive_indicators_individual_linearFit.pickle")

### Plot results

In [10]:
def cluster_dataframe(data, method="ward", row_cluster=True, col_cluster=True):
    from scipy.cluster.hierarchy import linkage, dendrogram

    if row_cluster:
        row_linkage = linkage(data, method=method, optimal_ordering=True)
        row_order = dendrogram(row_linkage, no_plot=True)['leaves']
    else:
        row_order = slice(None)

    if col_cluster:
        col_linkage = linkage(data.T, method=method,  optimal_ordering=True)
        col_order = dendrogram(col_linkage, no_plot=True)['leaves']

    else:
        col_order = slice(None)
        
    abs_max = data.abs().max(axis=None)
    
    return data.iloc[row_order, col_order]

def style_dataframe(data):
    max = data.loc[:,"Indicators"].abs().max(axis=None)
    styled_df = data.style.background_gradient(cmap="bwr", axis=None, vmin=-max, vmax=max, subset="Indicators")
    
    cm = sns.light_palette("green", reverse=True, as_cmap=True)
    max = data.loc[:,("Loss", "r2_score")].max(axis=None)
    min = data.loc[:,("Loss", "r2_score")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "r2_score")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Loss", "mae")].max(axis=None)
    min = data.loc[:,("Loss", "mae")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "mae")], vmin=min, vmax=max)
    
    
    display(styled_df)

#### Individual indicators

In [29]:
data = predictive_indicators_individual_linearFit.groupby(["Environmental concern dimension", "Year"]).mean() #average over all countries
#data = predictive_indicators_individual_linearFit.query("Country == 'United States'") #select specific country
#data = predictive_indicators_individual_linearFit.loc[("Awareness",2020,slice(None)),:] #select topic and year
#data = predictive_indicators_individual_linearFit #all data

cluster_rows = False
cluster_columns = True

data_clustered = cluster_dataframe(data.fillna(0).loc[:, ("Indicators", slice(None))], row_cluster=cluster_rows, col_cluster=cluster_columns)
cols = [*data_clustered.columns, *data.loc[:,("Loss", slice(None))].columns]
rows = data_clustered.index
style_dataframe(data.loc[rows,cols])

**Conclusions:**

1. **Political orientation** more important in recent years -> more polarized, environmental issues become political issues (left -> more pro-environment)
2. **Education level** important in predicting Will to make sacrifices and Efficacy of env. action (more educ. -> more pro-environment)
3. **Sex** correlated with awareness (females -> more aware)
4. **Age** slight correlation with all env. concern dimensions (older -> less concerned, more willing to make sacrifices)

#### Macro indicators

In [27]:
data = predictive_indicators_macro_linearFit
clustered_cols = [*cluster_dataframe(data.loc[:, ("Indicators", slice(None))], row_cluster=False).columns, *data.loc[:,("Loss", slice(None))].columns]
style_dataframe(data.loc[:,clustered_cols])

**Conclusions:**

1. **GDP** most important predictor -> less aware, but more willing to act (make sacrifices, believes in environmental action)

___
## Linear fit with basis expansion (needs fixing)

In [96]:
def fit_linear_basisExpansion(yX, weights=None):

    nFeatures = len(yX.columns) - 1
    
    #fill missing values with columns average
    yX = yX.fillna(yX.mean())
    #if whole column is missing fill with 0, then replace with NaN in output
    NaN_cols = [x for x in yX.iloc[:,1:].mean().isna().to_numpy() for i in range(3)]
    yX = yX.fillna(0)
    
    #standardize data
    scaler = MinMaxScaler()
    yX.loc[:] = scaler.fit_transform(yX)
    
    #basis expansion
    col_order = [idx for i in range(nFeatures) for idx in [1+i, 1+nFeatures+i, 1+2*nFeatures+i]]
    yX = pd.concat([yX.iloc[:,0], yX.iloc[:,1:], yX.iloc[:,1:]**2, np.log(yX.iloc[:,1:]+1e-20)], axis=1).iloc[:, [0,*col_order]]
    
    #standardize again
    new_scaler = MinMaxScaler()
    yX.loc[:] = new_scaler.fit_transform(yX)


    #for large datasets: single train-test split
    if len(yX) > 500:
        yX_train, yX_test = train_test_split(yX, test_size=0.2)

        if weights is not None:
            weights_train, weights_test = weights.loc[yX_train.index], weights.loc[yX_test.index]
        else:
            weights_train, weights_test = None, None

        #fit linear regression model
        model = LinearRegression(fit_intercept=True)
        model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

        #compute metrics        
        y_pred = (model.predict(yX_test.iloc[:,1:])-scaler.min_[0])/scaler.scale_[0]
        y_test = (yX_test.iloc[:,0]-scaler.min_[0])/scaler.scale_[0]
        
        r2_loss = r2_score(y_test, y_pred, sample_weight=weights_test)
        mae = mean_absolute_error(y_test, y_pred, sample_weight=weights_test)        

        return {"coefficients" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.coef_)], "losses" : [r2_loss, mae]}


    #for small datasets, leave one out cross validation
    else:
        
        y_pred = pd.Series(index=yX.index)#prepare dataframe for predictions

        for idx_num, idx_val in enumerate(yX.index):
            yX_train, yX_test = yX.loc[yX.index != idx_val, :], yX.loc[[idx_val], :]

            if weights is not None:
                weights_train, weights_test = weights.loc[weights.index != idx_val], weights.loc[weights.index == idx_val]
            else:
                weights_train, weights_test = None, None

            #fit linear regression model
            model = LinearRegression(fit_intercept=True)
            model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

            #get predictions
            y_pred.iloc[idx_num] = model.predict(yX_test.iloc[:,1:])

        #rescale back to original predictions data
        y_pred = (y_pred - scaler.min_[0])/scaler.scale_[0]
        y_test = yX.iloc[:,0]

        #compute metrics
        r2_loss = r2_score(y_test, y_pred, sample_weight=weights_test)
        mae = mean_absolute_error(y_test, y_pred, sample_weight=weights_test)        

        return {"coefficients" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.coef_)], "losses" : [r2_loss, mae]}

In [ ]:
#get indicators dataframes


##############################################################################################################################################################################################
indicators_macro = [("Country-wide indicators", ind) for ind in ["GHG_emissions", "PM25_exposure", "GDP_growth", "gini_index", "GDP_pcap_ppp", "water_withdrawals", "pesticides_use"]]
predictive_indicators_macro_linearFitBasisExp  = pd.DataFrame(columns=[("Indicators",ind, x) for _, ind in indicators_macro for x in ["x", "x^2", "log(x)"]]+[("Loss","r2_score", None),("Loss", "mae", None)]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered_grouped.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):    
    data = data_issp_numerical_clustered_grouped.loc[(year, slice(None)), [dimension, *indicators_macro]]
    res = fit_linear_basisExpansion(data)
    predictive_indicators_macro_linearFitBasisExp [(dimension[1], year)] = [*res["coefficients"], *res["losses"]]

predictive_indicators_macro_linearFitBasisExp = predictive_indicators_macro_linearFitBasisExp.T
predictive_indicators_macro_linearFitBasisExp.index = pd.MultiIndex.from_tuples(predictive_indicators_macro_linearFitBasisExp.index, names=["Environmental concern dimension", "Year"])
predictive_indicators_macro_linearFitBasisExp.columns = pd.MultiIndex.from_tuples(predictive_indicators_macro_linearFitBasisExp.columns)

##############################################################################################################################################################################################
indicators_individual = [("Respondent Data", ind) for ind in data_issp_numerical.loc[:, "Respondent Data"].drop(columns="Weight").columns]
predictive_indicators_individual_linearFitBasisExp = pd.DataFrame(columns=[("Indicators",ind, x) for _, ind in indicators_macro for x in ["x", "x^2", "log(x)"]]+[("Loss","r2_score", None),("Loss", "mae", None)]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):
    for country in data_issp_numerical_clustered.loc[year].index.get_level_values("Country").unique():
        data = data_issp_numerical_clustered.loc[(year, country, slice(None)), [dimension, *indicators_individual]]
        weights = data_issp_numerical_clustered.loc[(year, country, slice(None)), ("Respondent Data", "Weight")]
        res = fit_linear_basisExpansion(data, weights)
        predictive_indicators_individual_linearFitBasisExp[(dimension[1], year, country)] = [*res["coefficients"], *res["losses"]]
        
predictive_indicators_individual_linearFitBasisExp = predictive_indicators_individual_linearFitBasisExp.T
predictive_indicators_individual_linearFitBasisExp.index = pd.MultiIndex.from_tuples(predictive_indicators_individual_linearFitBasisExp.index, names=["Environmental concern dimension", "Year", "Country"])
predictive_indicators_individual_linearFitBasisExp.columns = pd.MultiIndex.from_tuples(predictive_indicators_individual_linearFitBasisExp.columns)



##############################################################################################################################################################################################
#dump results

#predictive_indicators_macro_linearFitBasisExp.to_pickle("../Saved_variables/predictive_indicators_macro_linearFitBasisExp.pickle")
#predictive_indicators_individual_linearFitBasisExp.to_pickle("../Saved_variables/predictive_indicators_individual_linearFitBasisExp.pickle")

In [ ]:
#quick load

predictive_indicators_macro_linearFitBasisExp = pd.read_pickle("../Saved_variables/predictive_indicators_macro_linearFitBasisExp.pickle")
predictive_indicators_individual_linearFitBasisExp = pd.read_pickle("../Saved_variables/predictive_indicators_individual_linearFitBasisExp.pickle")

### Plot results

In [100]:
def cluster_dataframe(data, method="ward", row_cluster=True, col_cluster=True):
    from scipy.cluster.hierarchy import linkage, dendrogram

    if row_cluster:
        row_linkage = linkage(data, method=method, optimal_ordering=True)
        row_order = dendrogram(row_linkage, no_plot=True)['leaves']
    else:
        row_order = slice(None)

    if col_cluster:
        col_linkage = linkage(data.T, method=method,  optimal_ordering=True)
        col_order = dendrogram(col_linkage, no_plot=True)['leaves']

    else:
        col_order = slice(None)
        
    abs_max = data.abs().max(axis=None)
    
    return data.iloc[row_order, col_order]

def style_dataframe(data):
    max = data.loc[:,"Indicators"].abs().max(axis=None)
    styled_df = data.style.background_gradient(cmap="bwr", axis=None, vmin=-max, vmax=max, subset="Indicators")
    
    cm = sns.light_palette("green", reverse=True, as_cmap=True)
    max = data.loc[:,("Loss", "r2_score")].max(axis=None)
    min = data.loc[:,("Loss", "r2_score")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "r2_score", None)], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Loss", "mae")].max(axis=None)
    min = data.loc[:,("Loss", "mae")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "mae", None)], vmin=min, vmax=max)
    
    
    display(styled_df)

#### Individual indicators

In [107]:
data = predictive_indicators_individual_linearFitBasisExp.groupby(["Environmental concern dimension", "Year"]).mean() #average over all countries
#data = predictive_indicators_individual_linearFitBasisExp.query("Country == 'United States'") #select specific country
#data = predictive_indicators_individual_linearFitBasisExp.loc[("Awareness",2020,slice(None)),:] #select topic and year
#data = predictive_indicators_individual_linearFitBasisExp #all data

cluster_rows = False
cluster_columns = True

data_clustered = cluster_dataframe(data.fillna(0).loc[:, ("Indicators", slice(None))], row_cluster=cluster_rows, col_cluster=cluster_columns)
cols = [*data_clustered.columns, *data.loc[:,("Loss", slice(None))].columns]
rows = data_clustered.index
style_dataframe(data.loc[rows,cols])

/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_788/3904516233.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
  max = data.loc[:,("Loss", "r2_score")].max(axis=None)
/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_788/3904516233.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  min = data.loc[:,("Loss", "r2_score")].min(axis=None)
/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_788/3904516233.py:31: PerformanceWarning: indexing past lexsort depth may impact performance.
  max = data.loc[:,("Loss", "mae")].max(axis=None)
/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_788/3904516233.py:32: PerformanceWarning: indexing past lexsort depth may impact performance.
  min = data.loc[:,("Loss", "mae")].min(axis=None)


#### Macro indicators

In [104]:
data = predictive_indicators_macro_linearFitBasisExp
clustered_cols = [*cluster_dataframe(data.loc[:, ("Indicators", slice(None))], row_cluster=False).columns, *data.loc[:,("Loss", slice(None))].columns]
style_dataframe(data.loc[:,clustered_cols])

/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_788/3904516233.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
  max = data.loc[:,("Loss", "r2_score")].max(axis=None)
/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_788/3904516233.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  min = data.loc[:,("Loss", "r2_score")].min(axis=None)
/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_788/3904516233.py:31: PerformanceWarning: indexing past lexsort depth may impact performance.
  max = data.loc[:,("Loss", "mae")].max(axis=None)
/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_788/3904516233.py:32: PerformanceWarning: indexing past lexsort depth may impact performance.
  min = data.loc[:,("Loss", "mae")].min(axis=None)


___
## Decision Trees (needs optimization)

In [21]:
def fit_trees(yX, weights=None):
        
    #fill missing values with columns average
    yX = yX.fillna(yX.mean())
    #if whole column is missing fill with 0, then replace with NaN in output
    NaN_cols = yX.iloc[:,1:].mean().isna().to_numpy()
    yX = yX.fillna(0)

    #for large datasets: single train-test split
    if len(yX) > 500:
        yX_train, yX_test = train_test_split(yX, test_size=0.2)

        if weights is not None:
            weights_train, weights_test = weights.loc[yX_train.index], weights.loc[yX_test.index]
        else:
            weights_train, weights_test = None, None
        
        #standardize data
        scaler = StandardScaler()
        yX_train.iloc[:,:] = scaler.fit_transform(yX_train)
        yX_test.iloc[:,:] = scaler.transform(yX_test)
        
        #fit linear regression model
        model = GradientBoostingRegressor()
        model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

        #compute metrics
        yX_pred = pd.DataFrame().reindex_like(yX_test)
        yX_pred.iloc[:,0] = model.predict(yX_test.iloc[:,1:])
        yX_pred.iloc[:,:] = scaler.inverse_transform(yX_pred)
        yX_test.iloc[:,:] = scaler.inverse_transform(yX_test)
        
        r2_loss = r2_score(yX_test.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights_test)
        mae = mean_absolute_error(yX_test.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights_test)
        

        return {"importance" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.feature_importances_)], "losses" : [r2_loss, mae]}


    #for small datasets, leave one out cross validation
    else:
        
        yX_pred = pd.DataFrame().reindex_like(yX) #prepare dataframe for predictions
        
        for idx_num, idx_val in enumerate(yX.index):
            yX_train, yX_test = yX.loc[yX.index != idx_val, :], yX.loc[[idx_val], :]

            if weights is not None:
                weights_train = weights.loc[weights.index != idx_val]
            else:
                weights_train = None

            #standardize data
            scaler = StandardScaler()
            yX_train.iloc[:,:] = scaler.fit_transform(yX_train)
            yX_test.iloc[:,:] = scaler.transform(yX_test)

            #fit linear regression model
            model = GradientBoostingRegressor()
            model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

            #get predictions
            yX_pred.iloc[idx_num,0] = model.predict(yX_test.iloc[:,1:])

        #rescale back to original predictions data
        yX_pred.iloc[:,:] = scaler.inverse_transform(yX_pred)

        #compute metrics
        r2_loss = r2_score(yX.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights)
        mae = mean_absolute_error(yX.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights)
        
        return {"importance" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.feature_importances_)], "losses" : [r2_loss, mae]}

In [ ]:
#get indicators dataframes


##############################################################################################################################################################################################
indicators_macro = [("Country-wide indicators", ind) for ind in ["GHG_emissions", "PM25_exposure", "GDP_growth", "gini_index", "GDP_pcap_ppp", "water_withdrawals", "pesticides_use"]]
predictive_indicators_macro_boostedTrees = pd.DataFrame(columns=[("Indicators",ind) for _, ind in indicators_macro]+[("Loss","r2_score"),("Loss", "mae")]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered_grouped.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):    
    data = data_issp_numerical_clustered_grouped.loc[(year, slice(None)), [dimension, *indicators_macro]]
    res = fit_trees(data)
    predictive_indicators_macro_boostedTrees[(dimension[1], year)] = [*res["importance"], *res["losses"]]

predictive_indicators_macro_boostedTrees = predictive_indicators_macro_boostedTrees.T
predictive_indicators_macro_boostedTrees.index = pd.MultiIndex.from_tuples(predictive_indicators_macro_boostedTrees.index, names=["Environmental concern dimension", "Year"])
predictive_indicators_macro_boostedTrees.columns = pd.MultiIndex.from_tuples(predictive_indicators_macro_boostedTrees.columns)


##############################################################################################################################################################################################
indicators_individual = [("Respondent Data", ind) for ind in data_issp_numerical.loc[:, "Respondent Data"].drop(columns="Weight").columns]
predictive_indicators_individual_boostedTrees = pd.DataFrame(columns=[("Indicators", ind) for _, ind in indicators_individual]+[("Loss","r2_score"),("Loss", "mae")]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):
    for country in data_issp_numerical_clustered.loc[year].index.get_level_values("Country").unique():
        data = data_issp_numerical_clustered.loc[(year, country, slice(None)), [dimension, *indicators_individual]]
        weights = data_issp_numerical_clustered.loc[(year, country, slice(None)), ("Respondent Data", "Weight")]
        res = fit_trees(data, weights)
        predictive_indicators_individual_boostedTrees[(dimension[1], year, country)] = [*res["importance"], *res["losses"]]
        
predictive_indicators_individual_boostedTrees = predictive_indicators_individual_boostedTrees.T
predictive_indicators_individual_boostedTrees.index = pd.MultiIndex.from_tuples(predictive_indicators_individual_boostedTrees.index, names=["Environmental concern dimension", "Year", "Country"])
predictive_indicators_individual_boostedTrees.columns = pd.MultiIndex.from_tuples(predictive_indicators_individual_boostedTrees.columns)



##############################################################################################################################################################################################
#dump results

#predictive_indicators_macro_boostedTrees.to_pickle("../Saved_variables/predictive_indicators_macro_boostedTrees.pickle")
#predictive_indicators_individual_boostedTrees.to_pickle("../Saved_variables/predictive_indicators_individual_boostedTrees.pickle")

In [27]:
#quick load

predictive_indicators_macro_boostedTrees = pd.read_pickle("../Saved_variables/predictive_indicators_macro_boostedTrees.pickle")
predictive_indicators_individual_boostedTrees = pd.read_pickle("../Saved_variables/predictive_indicators_individual_boostedTrees.pickle")

### Plot results

In [36]:
def cluster_dataframe(data, method="ward", row_cluster=True, col_cluster=True):
    from scipy.cluster.hierarchy import linkage, dendrogram

    if row_cluster:
        row_linkage = linkage(data, method=method, optimal_ordering=True)
        row_order = dendrogram(row_linkage, no_plot=True)['leaves']
    else:
        row_order = slice(None)

    if col_cluster:
        col_linkage = linkage(data.T, method=method,  optimal_ordering=True)
        col_order = dendrogram(col_linkage, no_plot=True)['leaves']

    else:
        col_order = slice(None)
        
    abs_max = data.abs().max(axis=None)
    
    return data.iloc[row_order, col_order]

def style_dataframe(data):
    max = data.loc[:,"Indicators"].max(axis=None)
    min = data.loc[:,"Indicators"].min(axis=None)
    styled_df = data.style.background_gradient(cmap="viridis", axis=None, vmin=min, vmax=max, subset="Indicators")
    
    cm = sns.light_palette("green", reverse=True, as_cmap=True)
    max = data.loc[:,("Loss", "r2_score")].max(axis=None)
    min = data.loc[:,("Loss", "r2_score")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "r2_score")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Loss", "mae")].max(axis=None)
    min = data.loc[:,("Loss", "mae")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "mae")], vmin=min, vmax=max)
    
    
    display(styled_df)

#### Individual indicators

In [37]:
data = predictive_indicators_individual_boostedTrees.groupby(["Environmental concern dimension", "Year"]).mean() #average over all countries
#data = predictive_indicators_individual_boostedTrees.query("Country == 'United States'") #select specific country
#data = predictive_indicators_individual_boostedTrees.loc[("Awareness",2010,slice(None)),:] #select topic and year
#data = predictive_indicators_individual_boostedTrees #all data

cluster_rows = False
cluster_columns = True

data_clustered = cluster_dataframe(data.fillna(0).loc[:, ("Indicators", slice(None))], row_cluster=cluster_rows, col_cluster=cluster_columns)
cols = [*data_clustered.columns, *data.loc[:,("Loss", slice(None))].columns]
rows = data_clustered.index
style_dataframe(data.loc[rows,cols])

#### Macro indicators

In [30]:
data = predictive_indicators_macro_boostedTrees
clustered_cols = [*cluster_dataframe(data.loc[:, ("Indicators", slice(None))], row_cluster=False).columns, *data.loc[:,("Loss", slice(None))].columns]
style_dataframe(data.loc[:,clustered_cols])

___
## TensorFlow multi-level linear model

- Fixed intercept (to start at average answer)
- Fixed slope for country-level predictors
- Slope varies by country for individual-level predictors

In [19]:
# Set up TensorFlow Model

# Extract unique country tags
unique_countries = data_issp_numerical_clustered.index.get_level_values("Country").unique().sort_values()

# Define inputs
macro_indicators = ["PM25 exposure", "GDP growth", "Gini index", "GDP per capita (Purchasing Power Parity)", "Pesticides use", "CO2 emissions per capita"]
micro_indicators = ['Sex', 'Age', 'Education level', 'Personal income', 'Living area', 'Political orientation', 'Religious beliefs']

# Define custom layer which uses different kernels for each country
class CustomDenseLayer(tf.keras.layers.Layer):
    def __init__(self, unique_countries, output_dim, **kwargs):
        self.unique_countries = unique_countries
        self.num_countries = len(unique_countries)
        self.output_dim = output_dim
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        # Create a kernel for each country
        self.kernels = {
            country: self.add_weight(
                name=f'kernel_{country}',
                shape=(input_shape[-1], self.output_dim),
                #initializer='zeros',
                trainable=True
            ) for country in self.unique_countries
        }
        super().build(input_shape)
    
    def call(self, inputs, country_indices):
        # Apply the appropriate kernel to each input based on the country index
        outputs = []
        for i, country in enumerate(self.unique_countries):
            mask = tf.cast(tf.equal(country_indices, i), tf.float32)
            masked_input = inputs * tf.expand_dims(mask, -1)
            output = tf.matmul(masked_input, self.kernels[country])
            outputs.append(output)
        result = tf.reduce_sum(tf.stack(outputs, axis=0), axis=0)[0]
        return tf.reshape(result, (-1, self.output_dim))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


class CustomSumLayer(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)

    def build(self, input_shape):
        self.bias = self.add_weight(
            name='bias',
            initializer='zeros',
            trainable=True,
            shape=(input_shape[-1])
        )
        super().build(input_shape)

    def call(self, input1, input2):
        return input1 + input2 + self.bias

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 1)

num_macro_ind = len(macro_indicators)
num_micro_ind = len(micro_indicators)

#create blank model (weights initialized to zero)
def new_model_tf():
    input_macro = tf.keras.layers.Input(shape=(num_macro_ind,), name="Macro_data")
    input_micro = tf.keras.layers.Input(shape=(num_micro_ind,), name="Micro_data")
    input_country_indices = tf.keras.layers.Input(shape=(1,), dtype=tf.int32, name="Country_indices")
    
    # Define shared weight layer for macro indicators
    #macro_dense_layer = tf.keras.layers.Dense(1, kernel_initializer="zeros", use_bias=False, name="Macro_dense_layer")(input_macro)
    macro_dense_layer = tf.keras.layers.Dense(1, use_bias=False, name="Macro_dense_layer")(input_macro)
    
    # Define custom weight layer for micro indicators
    custom_dense_layer = CustomDenseLayer(unique_countries, 1, name="Micro_custom_layer")
    micro_custom_layer = custom_dense_layer(input_micro, input_country_indices)
    
    # Define the output layer
    custom_sum_layer = CustomSumLayer(name="Get_prediction")
    output = custom_sum_layer(macro_dense_layer, micro_custom_layer)
    
    # Create the model
    model = tf.keras.models.Model(inputs=[input_macro, input_micro, input_country_indices], outputs=output, name="Model")
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')

    return model

In [20]:
# Preprocessing data for the model
def preprocess_data_tf(df, year, macro_ind, micro_ind, target):
    # select year
    df = df.loc[year]
    
    # fill missing individual values with country average and macro indicator with year average. Return which countries and indicators are missing for further processing
    # if individual indicator is missing for whole country replace with 0.
    missing_values = df.groupby("Country").mean().map(lambda x: np.isnan(x)) # mark which countries have missing data
    
    df_micro_filled = df.loc[:,("Respondent Data", slice(None))].fillna(df.loc[:,("Respondent Data", slice(None))].groupby("Country").mean()) #fill individual missing data with country average
    df_micro_filled = df_micro_filled.fillna(0) #if whole individual indicator is missing fill with 0 (indicator for country will be marked as true in missing_values dataframe)
    
    df_macro_filled = df.loc[:,("Country-wide indicators", slice(None))].fillna(df.loc[:,("Country-wide indicators", slice(None))].mean()) #fill macro indicator with average
    
    df_filled = pd.concat([df_micro_filled, df_macro_filled, df.loc[:,("Environmental concern dimension", slice(None))]], axis=1) #get dataframe

    #train-test split
    df_train, df_test = train_test_split(df_filled, test_size=0.2)    

    #normalization
    scaler = StandardScaler()
    df_train.iloc[:] = scaler.fit_transform(df_train)
    df_test.iloc[:] = scaler.transform(df_test)
    

    #split data into macro and micro indicators
    X_macro_train = df_train.loc[:, df_train.columns.get_level_values("Variable Name").isin(macro_ind)].values
    X_micro_train = df_train.loc[:, df_train.columns.get_level_values("Variable Name").isin(micro_ind)].values
    y_train = df_train.loc[:, ("Environmental concern dimension", target)].values
    country_indices_train = df_train.index.get_level_values("Country").to_series().map({country: i for i, country in enumerate(unique_countries)}).values
    
    X_macro_test = df_test.loc[:, df_test.columns.get_level_values("Variable Name").isin(macro_ind)].values
    X_micro_test = df_test.loc[:, df_test.columns.get_level_values("Variable Name").isin(micro_ind)].values
    y_test = df_test.loc[:, ("Environmental concern dimension", target)].values
    country_indices_test = df_test.index.get_level_values("Country").to_series().map({country: i for i, country in enumerate(unique_countries)}).values
    
    
    return [X_macro_train, X_micro_train, country_indices_train], y_train, [X_macro_test, X_micro_test, country_indices_test], y_test, missing_values

In [22]:
# Fitting Model

index_micro = pd.MultiIndex.from_product([
    ['Awareness', 'Will to make sacrifices','Efficacy of environmental action'],
    [1993, 2000, 2010, 2020],
    unique_countries],
    names = ["Environmental Concern Dimension", "Year", "Country"]
                                )
index_macro = pd.MultiIndex.from_product([
    ['Awareness', 'Will to make sacrifices', 'Efficacy of environmental action'],
    [1993, 2000, 2010, 2020]],
    names = ["Environmental Concern Dimension", "Year"]
                                )



predictive_indicators_individual_tensorflowModel = pd.DataFrame(index=index_micro, columns=pd.Index(micro_indicators, name="Indicators"))
predictive_indicators_macro_tensorflowModel = pd.DataFrame(index=index_macro, columns=pd.Index(macro_indicators, name="Indicators"))
losses = pd.DataFrame(index = index_micro, columns=pd.Index(["r2", "mae", "mse"], name="Losses"))

for target in ['Awareness', 'Will to make sacrifices', 'Efficacy of environmental action']:
    for year in [1993, 2000, 2010, 2020]:
        print(f"Training custom TensorFlow model for '{target}' in {year}\n")    
        X_train, y_train, X_test, y_test, missing_data = preprocess_data_tf(data_issp_numerical_clustered, year, macro_indicators, micro_indicators, target)       
        model = new_model_tf()

        #fit and get weights
        earlystop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=5, restore_best_weights=True, start_from_epoch=5)
        
        model.fit(X_train, y_train, epochs=50, batch_size=None, verbose=3, callbacks=[earlystop])
        predictive_indicators_individual_tensorflowModel.loc[(target, year, slice(None)),:] = pd.DataFrame.from_dict(model.get_layer("Micro_custom_layer").kernels).map(lambda x: x.numpy()[0]).T.values
        predictive_indicators_macro_tensorflowModel.loc[(target, year),:] = pd.DataFrame.from_dict(model.get_layer("Macro_dense_layer").weights).map(lambda x: x.numpy()[0]).values

        #get predictions and evaluate model
        y_pred = model.predict(X_test, verbose=0).squeeze()

        for i, country in enumerate(unique_countries):
            y_test_country = [y for k,y in enumerate(y_test) if X_test[2][k] == i]
            y_pred_country = [y for k,y in enumerate(y_pred) if X_test[2][k] == i]

            if len(y_test_country) > 0:
                r2_loss = r2_score(y_test_country, y_pred_country)
                mae = mean_absolute_error(y_test_country, y_pred_country)
                mse = mean_squared_error(y_test_country, y_pred_country)
                losses.loc[(target, year, country),:] = r2_loss, mae, mse

        del(model)
        
        print("\n\n")


predictive_indicators_individual_tensorflowModel = pd.concat([predictive_indicators_individual_tensorflowModel, losses], axis=1, keys=["Indicators", "Loss"])
predictive_indicators_macro_tensorflowModel = pd.concat([predictive_indicators_macro_tensorflowModel, losses.groupby(["Environmental Concern Dimension", "Year"]).mean()], axis=1, keys=["Indicators", "Loss"])

index = [idx in data_issp_numerical_clustered.index.droplevel(2) for idx in predictive_indicators_individual_tensorflowModel.index.droplevel(0)]
predictive_indicators_individual_tensorflowModel = predictive_indicators_individual_tensorflowModel.loc[index]

Training custom tensorflow model for 'Awareness' in 1993

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50



Training custom tensorflow model for 'Awareness' in 2000

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50



Training custom tensorflow model for 'Awareness' in 2010

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50



Training custom tensorflow model for 'Awareness' in 2020

Epoch 1/50
Epoch 2/50
Epoch 3/

In [ ]:
#Save data
predictive_indicators_individual_tensorflowModel.to_pickle("../Saved_variables/predictive_indicators_individual_tensorflow.pickle")
predictive_indicators_macro_tensorflowModel.to_pickle("../Saved_variables/predictive_indicators_macro_tensorflow.pickle")

___

In [7]:
#quick load
predictive_indicators_individual_tensorflowModel = pd.read_pickle("../Saved_variables/predictive_indicators_individual_tensorflow.pickle")
predictive_indicators_macro_tensorflowModel = pd.read_pickle("../Saved_variables/predictive_indicators_macro_tensorflow.pickle")

___
### Plot results

In [23]:
def cluster_dataframe(data, method="ward", row_cluster=True, col_cluster=True):
    from scipy.cluster.hierarchy import linkage, dendrogram

    if row_cluster:
        row_linkage = linkage(data, method=method, optimal_ordering=True)
        row_order = dendrogram(row_linkage, no_plot=True)['leaves']
    else:
        row_order = slice(None)

    if col_cluster:
        col_linkage = linkage(data.T, method=method,  optimal_ordering=True)
        col_order = dendrogram(col_linkage, no_plot=True)['leaves']

    else:
        col_order = slice(None)
        
    abs_max = data.abs().max(axis=None)
    
    return data.iloc[row_order, col_order]

def style_dataframe(data):
    max = data.loc[:,"Indicators"].abs().max(axis=None)
    styled_df = data.style.background_gradient(cmap="bwr", axis=None, vmin=-max, vmax=max, subset="Indicators")
    
    cm = sns.light_palette("green", reverse=True, as_cmap=True)
    max = data.loc[:,("Loss", "r2")].max(axis=None)
    min = data.loc[:,("Loss", "r2")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "r2")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Loss", "mae")].max(axis=None)
    min = data.loc[:,("Loss", "mae")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "mae")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Loss", "mse")].max(axis=None)
    min = data.loc[:,("Loss", "mse")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "mse")], vmin=min, vmax=max)

    
    #display(styled_df)
    return styled_df

#### Individual indicators

In [30]:
data = predictive_indicators_individual_tensorflowModel.groupby(["Environmental Concern Dimension", "Year"]).mean() #average over all countries
#data = predictive_indicators_individual_tensorflowModel.query("Country == 'United States'") #select specific country
#data = predictive_indicators_individual_tensorflowModel.loc[("Awareness",1993,slice(None)),:] #select topic and year
#data = predictive_indicators_individual_tensorflowModel #all data

cluster_rows = False
cluster_columns = True

data_clustered = cluster_dataframe(data.fillna(0).loc[:, ("Indicators", slice(None))], row_cluster=cluster_rows, col_cluster=cluster_columns)
cols = [*data_clustered.columns, *data.loc[:,("Loss", slice(None))].columns]
rows = data_clustered.index

#print(style_dataframe(data.loc[rows,cols]).to_latex(environment="longtable", convert_css=True).replace("\color[HTML]{000000}",""))

style_dataframe(data.loc[rows,cols])

/var/folders/mv/3xxn6x011nj1xbbrht5z6r1m0000gn/T/ipykernel_921/1961632180.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_clustered = cluster_dataframe(data.fillna(0).loc[:, ("Indicators", slice(None))], row_cluster=cluster_rows, col_cluster=cluster_columns)


#### Macro indicators

In [25]:
data = predictive_indicators_macro_tensorflowModel
data.columns = data.columns.map(lambda x: (x[0],{"GDP per capita (Purchasing Power Parity)":"GDP p.cap.", "CO2 emissions per capita" : "CO2 emiss. p.cap."}.get(x[1],x[1])))
clustered_cols = [*cluster_dataframe(data.loc[:, ("Indicators", slice(None))], row_cluster=False).columns, *data.loc[:,("Loss", slice(None))].columns]

#print(style_dataframe(data.loc[:,clustered_cols]).to_latex(convert_css=True).replace("\color[HTML]{000000}",""))

style_dataframe(data.loc[:,clustered_cols])

___
## StatsModel Multi-Level linear model

In [174]:
# Preprocessing data
def preprocess_data_mixedLM(df, year, macro_ind, micro_ind, target):
    # select year
    df = df.loc[year]
    
    # fill missing individual values with country average and macro indicator with year average. Return which countries and indicators are missing for further processing
    # if individual indicator is missing for whole country replace with 0.
    missing_values = df.groupby("Country").mean().map(lambda x: np.isnan(x)) # mark which countries have missing data
    
    df_micro_filled = df.loc[:,("Respondent Data", slice(None))].fillna(df.loc[:,("Respondent Data", slice(None))].groupby("Country").mean()) #fill individual missing data with country average
    df_micro_filled = df_micro_filled.fillna(0) #if whole individual indicator is missing fill with 0 (indicator for country will be marked as true in missing_values dataframe)
    
    df_macro_filled = df.loc[:,("Country-wide indicators", slice(None))].fillna(df.loc[:,("Country-wide indicators", slice(None))].mean()) #fill macro indicator with average
    
    df_filled = pd.concat([df_micro_filled, df_macro_filled, df.loc[:,("Environmental concern dimension", slice(None))]], axis=1) #get dataframe

    #select features
    df_filled = df_filled.loc[:,df_filled.columns.get_level_values("Variable Name").isin(macro_ind+micro_ind+[target])]
    
    #train-test split
    df_train, df_test = train_test_split(df_filled, test_size=0.2)    

    #normalization
    scaler = StandardScaler()
    df_train.iloc[:] = scaler.fit_transform(df_train)
    df_test.iloc[:] = scaler.transform(df_test)

    #set up country column for mixed model class
    df_train[("Class", "Country")] = df_train.index.get_level_values("Country")
    df_train = df_train.reset_index(drop=True)

    df_test[("Class", "Country")] = df_test.index.get_level_values("Country")
    df_test = df_test.reset_index(drop=True)

    #drop columns label
    df_train.columns = df_train.columns.droplevel("Variable Type")
    df_test.columns = df_test.columns.droplevel("Variable Type")
    
    return df_train, df_test

In [300]:
# Fitting models and getting parameters and metrics

import statsmodels.formula.api as smf
macro_indicators = ["PM25 exposure", "GDP growth", "Gini index", "GDP per capita (Purchasing Power Parity)", "Pesticides use", "CO2 emissions per capita"]
micro_indicators = ['Sex', 'Age', 'Education level', 'Personal income', 'Living area', 'Political orientation', 'Religious beliefs']

index_mixedLM = pd.MultiIndex.from_product([
    ['Awareness', 'Will to make sacrifices', 'Efficacy of environmental action'],
    [1993, 2000, 2010, 2020]],
    names = ["Environmental Concern Dimension", "Year"]
                                )
predictive_indicators_mixedLM = pd.DataFrame(index=index_mixedLM, columns = pd.Index(macro_indicators+micro_indicators, name="Indicators"))
metrics_mixedLM = pd.DataFrame(index=index_mixedLM, columns = pd.Index(["r2", "mse", "mae", "Residual variance", "ICC", "Converged"], name="Metrics"))

cols_renamed_dict = {ind: ind.replace(" ", "_") for ind in macro_indicators+micro_indicators+['Awareness', 'Will to make sacrifices', 'Efficacy of environmental action']}
cols_renamed_dict.update({"GDP per capita (Purchasing Power Parity)" : "GDP_per_capita_PPP"})    

for target in ['Awareness', 'Will to make sacrifices', 'Efficacy of environmental action']:
    for year in [1993, 2000, 2010, 2020]:
        print(f"Training mixed-effects linear model for '{target}' in {year}\n")    
        data_train, data_test = preprocess_data_mixedLM(data_issp_numerical_clustered, year, macro_indicators, micro_indicators, target)

        #rename columns (no spaces or parenthesis)
        data_train.columns = data_train.columns.map(lambda x: cols_renamed_dict.get(x,x))
        data_test.columns = data_test.columns.map(lambda x: cols_renamed_dict.get(x,x))
        
        macro_indicators_ = [cols_renamed_dict[ind] for ind in macro_indicators]
        micro_indicators_ = [cols_renamed_dict[ind] for ind in micro_indicators]
        target_ = cols_renamed_dict[target]
        
        # Fit the random slope model
        model = smf.mixedlm(
            f"{target_} ~ {' + '.join(macro_indicators_)} + {' + '.join(micro_indicators_)}",
            data_train, 
            groups=data_train["Country"], 
            re_formula=f"1 + {' + '.join(micro_indicators_)}"
        )
        
        result = model.fit(method=["lbfgs"])
    
        #get parameters
        params = result.fe_params.drop("Intercept")
        params.index = params.index.map({k:i for i,k in cols_renamed_dict.items()})
    
        #get metrics
        res_var = result.scale
        intra_class_var = result.cov_re.values.diagonal().sum()
        icc = intra_class_var / (intra_class_var + 1)

        
        y_pred = result.fittedvalues
        y_true = data_train[target_]
        
        r2_loss = r2_score(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
    
        #save params+metrics
        predictive_indicators_mixedLM.loc[(target, year),:] = params
        if result.converged:
            metrics_mixedLM.loc[(target, year),:] = [r2_loss, mse, mae, res_var, icc, True]
        else:
            metrics_mixedLM.loc[(target, year),:] = [r2_loss, mse, mae, res_var, icc, False]

        del(model, result, params, res_var, intra_class_var, icc, r2_loss, mse, mae)

predictive_indicators_mixedLM = pd.concat([predictive_indicators_mixedLM, metrics_mixedLM], axis=1, keys=["Indicators", "Metrics"])

newLabels = {("Indicators", item) : ("Individual indicators", item) for item in micro_indicators}
newLabels.update({("Indicators", item) : ("Country-level indicators", item) for item in macro_indicators})
predictive_indicators_mixedLM.columns = predictive_indicators_mixedLM.columns.map(lambda x: newLabels.get(x,x))
del(newLabels)

#predictive_indicators_mixedLM.to_pickle("../Saved_variables/predictive_indicators_mixedLM.pickle")

del(metrics_mixedLM)

Training mixed-effects linear model for 'Awareness' in 1993



/Users/Edoardo/anaconda3/envs/tesi/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/Edoardo/anaconda3/envs/tesi/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


Training mixed-effects linear model for 'Awareness' in 2000



/Users/Edoardo/anaconda3/envs/tesi/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Training mixed-effects linear model for 'Awareness' in 2010



...

/Users/Edoardo/anaconda3/envs/tesi/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


Training mixed-effects linear model for 'Efficacy of environmental action' in 2010



/Users/Edoardo/anaconda3/envs/tesi/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Training mixed-effects linear model for 'Efficacy of environmental action' in 2020



/Users/Edoardo/anaconda3/envs/tesi/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


___

In [ ]:
# quick load
predictive_indicators_mixedLM = pd.read_pickle("../Saved_variables/predictive_indicators_mixedLM.pickle")

___
### Plot results

In [302]:
def cluster_dataframe(data, method="ward", row_cluster=True, col_cluster=True):
    from scipy.cluster.hierarchy import linkage, dendrogram

    if row_cluster:
        row_linkage = linkage(data, method=method, optimal_ordering=True)
        row_order = dendrogram(row_linkage, no_plot=True)['leaves']
    else:
        row_order = slice(None)

    if col_cluster:
        col_linkage = linkage(data.T, method=method,  optimal_ordering=True)
        col_order = dendrogram(col_linkage, no_plot=True)['leaves']

    else:
        col_order = slice(None)
    
    return data.iloc[row_order, col_order]

def style_dataframe_mixedLM(data, dropConverged=False):

    if dropConverged == False:
        #convert convergence column
        data.loc[:,("Metrics", "Converged")] = data.loc[:,("Metrics", "Converged")].map(lambda x: "$\checkmark$" if x else r"$\times$")
    else:
        data = data.drop(columns=[("Metrics", "Converged")])
    
    try:
        max = data.loc[:,["Individual indicators", "Country-level indicators"]].abs().max(axis=None)
    except:
        try:
            max = data.loc[:,["Individual indicators"]].abs().max(axis=None)
        except:
            pass
        try:
            max = data.loc[:,["Country-level indicators"]].abs().max(axis=None)
        except:
            pass

    styled_df = data.style
    
    if "Individual indicators" in data.columns.get_level_values(0):
        styled_df = styled_df.background_gradient(cmap="bwr", axis=None, vmin=-max, vmax=max, subset=["Individual indicators"])
    if "Country-level indicators" in data.columns.get_level_values(0):
        styled_df = styled_df.background_gradient(cmap="bwr", axis=None, vmin=-max, vmax=max, subset=["Country-level indicators"])
    
    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Metrics", "Residual variance")].max(axis=None)
    min = data.loc[:,("Metrics", "Residual variance")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Metrics", "Residual variance")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=True, as_cmap=True)
    max = data.loc[:,("Metrics", "r2")].max(axis=None)
    min = data.loc[:,("Metrics", "r2")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Metrics", "r2")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Metrics", "mse")].max(axis=None)
    min = data.loc[:,("Metrics", "mse")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Metrics", "mse")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Metrics", "mae")].max(axis=None)
    min = data.loc[:,("Metrics", "mae")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Metrics", "mae")], vmin=min, vmax=max)

    cm = sns.light_palette("lime", reverse=False, as_cmap=True)
    max = data.loc[:,("Metrics", "ICC")].max(axis=None)
    min = data.loc[:,("Metrics", "ICC")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Metrics", "ICC")], vmin=min, vmax=max)

    if dropConverged == False:
        apply_hatch = lambda row: ['background-image: repeating-linear-gradient(35deg, rgba(0,0,0,0.3) 0, rgba(0,0,0,0.3) 1px, transparent 1px, transparent 5px);'] * len(row) if row[("Metrics", "Converged")] == r"$\times$" else [''] * len(row)
        styled_df = styled_df.apply(apply_hatch, axis=1)

    
    return styled_df

In [303]:
data1 = cluster_dataframe(predictive_indicators_mixedLM.loc[:, ("Country-level indicators", slice(None))], row_cluster=False)
data2 = cluster_dataframe(predictive_indicators_mixedLM.loc[:, ("Individual indicators", slice(None))], row_cluster=False)

cols = [*data1.columns, *data2.columns, *predictive_indicators_mixedLM.loc[:,["Metrics"]].columns]
#cols = [*data1.columns, *predictive_indicators_mixedLM.loc[:,["Metrics"]].columns]
#cols = [*data2.columns, *predictive_indicators_mixedLM.loc[:,["Metrics"]].columns]

#print(style_dataframe_mixedLM(predictive_indicators_mixedLM.loc[:,cols], dropConverged=True).to_latex(convert_css=True).replace("\color[HTML]{000000}",""))

style_dataframe_mixedLM(predictive_indicators_mixedLM.loc[:,cols])

___